Заполните данные о необходимых переменных:

In [1]:
Today = '29.02.2024'
Mounth = 'Feb'

Group = 'EUROCHEM'

In [2]:
import os
files_list = os.listdir('.')
work_files_list = [x for x in files_list if ('.xlsx' in x and '~' not in x and 'ODGS' in x and '02_' in x)] # Месяц
Data_file = [x for x in work_files_list if 'ECh' in x][0] # Холдинг
Sheet_in_data_file = 'Export Worksheet'

from datetime import date
Output_file = "_".join([str(date.today()), Group, 'SAP', 'ARAP', f'{Mounth}.xlsx'])
Sheet_in_output_file = 'ARAP'

work_files_list

['ODGS_2024_02_M_ECh.xlsx']

Открываю следующие файлы:

In [3]:
import pandas as pd
import numpy as np

data_export = pd.read_excel(Data_file, sheet_name = Sheet_in_data_file)

In [4]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list

In [21]:
data_work = data_export.query('holding != "EXTERNAL"').reset_index(drop=True)
# фильтрация по перемнной Group

data_work['org_code'] = merge_Mapping(data_work, col='orgName')
data_work['CompCode_concat'] = data_work['companyCode']
data_work.loc[data_work.companyCode.isna(), 'CompCode_concat'] = data_work['org_code'].replace({'External': ''})
data_work['CompMap'] = merge_SalesUnits(data_work, col='CompCode_concat', merge_col='ocpSegment')

data_work['CtyCode'] = merge_Mapping(data_work, col='customerName').fillna('External')
data_work['CtyMap'] = merge_SalesUnits(data_work, col='CtyCode', merge_col='ocpSegment').fillna('External')

data_work['AR_flag'] = data_work['accountPurpose']\
                                            .isin(['Trade receivables', 'Other receivables', 'Provisions'])\
                                            .astype(int)
data_work['AP_flag'] = data_work['accountPurpose']\
                                            .isin(['Trade payables', 'Other payables'])\
                                            .astype(int)

data_work['Amount_USD'] = data_work['presentAmountUSD'].replace({np.NaN : 0})
data_work['Amount_AR_USD'] = data_work['Amount_USD'] * data_work['AR_flag']
data_work['Amount_AP_USD'] = data_work['Amount_USD'] * data_work['AP_flag']

data_work = Period(data_work, day_for_count=Today, col_with_date='dueDate')

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:41: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  day = pd.to_datetime(day_for_count)


Записываю данные в output file:

In [22]:
data_print = data_work[['customerName', 'CompCode_concat', 'currencyCode', 'sourceSystem',\
                    'accountPurpose', 'CompMap', 'CtyCode', 'CtyMap','Amount_USD',\
                    'AR_flag', 'Amount_AR_USD', 'AP_flag', 'Amount_AP_USD', 'Days', 'Period']]

Проверка на merge:

In [23]:
check_data = data_work.loc[(data_work['internalExternalEN'] == 'Internal') & (data_work['CtyMap'] == 'External'),\
                                                    ['customerName', 'CtyCode', 'CtyMap']].drop_duplicates()
check_data
# check_data.to_excel('manual_map_ARAP_ECH.xlsx', index=False)

# check_data2 = data_work.loc[(data_work['internalExternalEN'] == 'Internal') & (data_work['CompMap'] == 'External'),\
#                                                     ['companyCode','CompMap']]
# check_data2

,customerName,CtyCode,CtyMap
59136,Благотворительный фонд Андрея Мельниченко,External,External
59945,"ООО ""Германий и приложения""",External,External
69554,EuroChem Management Middle East FZE,AEMNG,External
69699,EUROCHEM LOUISIANA LLC,External,External
90933,Отдел экспедирования грузов и агентирования су...,External,External


In [24]:
BS_data = data_work[(data_work.accountPurpose.str.contains('Trade')) | (data_work.accountPurpose.str.contains('Other'))].reset_index(drop=True)
BS_data['Comp_hold'] = merge_SalesUnits(BS_data, col='CompCode_concat', merge_col='holding')
BS_data['Cpty_hold'] = merge_SalesUnits(BS_data, col='CtyCode', merge_col='holding')

BS_data = BS_data[(BS_data.Comp_hold == Group) & (BS_data.Cpty_hold == Group)]
BS_data = BS_data[BS_data.CompMap != 'External']
BS_data = BS_data[['CompMap', 'CtyMap', 'Amount_USD']]
BS_pivot = pd.pivot_table(BS_data, values='Amount_USD', index='CompMap', aggfunc='sum', columns='CtyMap',)
BS_pivot

CtyMap,EURPROD,EURSALES,NAMSALES,RUFERT,SAMSALES,SPV,TRADING
CompMap,,,,,,,
EURPROD,13.06,1.693441e+06,NaN,-5.262765e+06,1.167090e+05,NaN,7.110114e+07
EURSALES,-921524.54,3.785075e+06,NaN,-3.321980e+07,2.168295e+06,NaN,-2.723685e+08
NAMSALES,NaN,NaN,NaN,NaN,NaN,NaN,-4.571481e+08
RUFERT,1666.83,7.245540e+05,NaN,2.049248e+06,NaN,NaN,2.672465e+09
SAMSALES,NaN,-2.168295e+06,NaN,NaN,-8.133124e+05,NaN,-3.512417e+07
TRADING,-58596163.69,2.744558e+08,4.571421e+08,-2.337926e+09,1.966901e+09,-5316247.41,3.963384e+07


In [9]:
# assert 1==2

In [25]:
BS_pivot.to_excel(Output_file, sheet_name='BS_to_BS')
new_list(data_print, Output_file, Sheet_in_output_file)